In [1]:
import os
import boto3
import pickle
import numpy
import music_tag
import random
import json
from pathlib import Path

from music_utils import *

import numpy as np
from pymilvus import connections, utility
from pymilvus import Collection, DataType, FieldSchema, CollectionSchema

In [2]:
DATASET = "Music_Dataset"
BUCKET = "awsembeddings"

In [3]:
def get_list_of_all_filepaths_in_S3(bucket):
    s3_client = boto3.client('s3')
    try:
        contents = s3_client.list_objects(Bucket=bucket)
        return [content['Key'] for content in contents['Contents']]

    except Exception as e:
        pass
    return None


In [4]:
s3_client = boto3.client('s3') 

list_of_files_in_S3 = get_list_of_all_filepaths_in_S3(BUCKET)
print(len(list_of_files_in_S3))



174


In [5]:
import random
rdm_from_s3 = random.choice(list_of_files_in_S3)
print(rdm_from_s3)

uploads/Music_Dataset/The_Matrix_OST/01_Marilyn_Manson_-_Rock_Is_Dead.mp3


In [6]:
URI = "https://in03-efa63c0579a14a1.api.gcp-us-west1.zillizcloud.com"
TOKEN = "e58021f476f7b39e5d84eb5c804e27bfec1a7fb89b6e01f7560ac57877be699b9b1f109a2ba8fabefd2fa26f2efab109ebdd79f0"

connections.connect("default",
                    uri=URI,
                    token=TOKEN)
print(f"Connecting to DB: {URI}")
print(utility.list_collections())

collection_512 = Collection("embeddings_512")

Connecting to DB: https://in03-efa63c0579a14a1.api.gcp-us-west1.zillizcloud.com
['predictions_87', 'embeddings_512']


In [7]:
query_path = rdm_from_s3.replace("uploads/", "")

In [8]:
entities = collection_512.query(
    expr=f'path == "{query_path}"',
    output_fields=["*"]
)
embedding_512 = entities[0]['embedding']


In [13]:
query_artist = entities[0]['artist']
query_album = entities[0]['album']
query_title = entities[0]['title']
query_top3_genres = entities[0]['top_5_genres'][:3]

print(query_artist)
print(query_album)
print(query_title)
print(query_top3_genres)

Marilyn Manson
The Matrix
Rock Is Dead
['rock', 'alternative', 'indie']


In [14]:
bigresult = collection_512.search(
    data=[embedding_512],
    anns_field="embedding",
    param={"nprobe": 16},
    limit=200,
    offset=1,
    output_fields=["*"],
)

print(f'{"Title":<30} | {"Artist":<25} | {"Top 5 Genres":<25} | {"Path":<50}')
print('-' * 145)

already_proposed_artits = []
count = 0
for result in bigresult[0]:
    if count >= 20:
        break
    already_proposed_artits.append(result.artist)
    count += 1
    print(f"{result.title[:29]:<30} | {result.artist[:39]:<25} | {', '.join(result.top_5_genres[:2]):<25} | {result.path[:49]:<50} | ")

Title                          | Artist                    | Top 5 Genres              | Path                                              
-------------------------------------------------------------------------------------------------------------------------------------------------
Dragula [Hot Rod Herman Remix  | Rob Zombie                | electronic, rock          | Music_Dataset/The_Matrix_OST/08_Rob_Zombie_-_Drag  | 
Wake Up                        | Rage Against The Machine  | rock, alternative         | Music_Dataset/The_Matrix_OST/13_Rage_Against_The_  | 
Aimable à souhait              | Mass Hysteria             | rock, metal               | Music_Dataset/Mass_Hysteria/03 - Aimable à souhai  | 
Grave Robbing U.S.A.           | Murderdolls               | rock, metal               | Music_Dataset/Murderdolls/Murderdolls_-_Grave_Rob  | 
N.P.F.S.                       | No Place For Soul         | rock, electronic          | Music_Dataset/No_Place_For_Soul/09 N.P.F.S.mp3     | 